<a href="https://colab.research.google.com/github/LUMC/EfDS_RelDB_SQL/blob/main/orm_practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Object Relational Mapper

## Goals

The overall goal of this session is to learn how to provide an object oriented interface to a database.  
Since this is a complex task, specialized libraries help to provide such functionality.  
Here, we will use a part of the `SQLAlchemy` library called [Object Relational Mapper](https://docs.sqlalchemy.org/en/14/orm/index.html) (ORM).

This lecture describes a growing code and is intended to be followed sequentially. Subsequent steps have the following goals:
- Create a class corresponding to a single database table (relations are ignored).
- Understand the concepts of engine, session, transaction, commit, rollback.
- Query the database and get results as objects representing rows. Build simple and more complex queries.
- Extend the class with a method for nicer content printing.
- Create another class for another database table. Learn how do declare foreign keys and add methods representing relations.
- Add new content to the database. Commit. Show the newly added content.

*Note:* [The PYSheet cheatsheet](https://www.pythonsheets.com/notes/python-sqlalchemy.html) is useful once you understand the basic SQLAlchemy and Object Relational Mapper concepts.

Below there is (again) the diagram from [the SQLite Tutorial](https://www.sqlitetutorial.net) of the `chinook` database.

![chinook scheme](https://www.sqlitetutorial.net/wp-content/uploads/2015/11/sqlite-sample-database-color.jpg)

We will implement classes providing access to a small part of the database:
- We want classes `Album` (a row of the table `albums`) or `Track` (a row of the table `tracks`).
- The class `Album` should have a field like `title`.
- The class `Album` should have a method `tracks` which returns respective `Track` objects for "the current" `Album` object.
- When a field is changed, the database should get updated.

## Preparation


The following tools/sources will be used and they need to get installed:
- `chinook.db` is the database (a file in `SQLite` database format)
- `SQLAlchemy` is the Python library which:
    - provides unified SQL access to databases of many formats, including `SQLite`
    - provides Object Relational Mapper functionality


### Download of the database

The following lines download a compressed file `chinook.zip` and unpack it leading to a SQLite database file `chinook.db`.  
*Note:* In this session we will modify the database, so the database file will change and some examples may not work anymore. In that case, you can download the database again.

In [9]:
import urllib.request    # needed for download of the example database
import shutil            # needed for unziping of the example database
import os                # for removing a file


In [8]:

# urllib.request.urlretrieve("https://www.sqlitetutorial.net/wp-content/uploads/2018/03/chinook.zip", "chinook.zip")
# shutil.unpack_archive("chinook.zip")
# os.remove("chinook.zip")

URLError: <urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1000)>

### SQLAlchemy toolkit

The demonstrations will use multiple functions and classes of the SQLAlchemy library.

In [10]:
import sqlalchemy as sa
import sqlalchemy.orm as orm

### Database connection and transaction

When using the SQLAlchemy library, the following objects are used to interact with the database. 
- `engine`: Provides a main connection to a database.
- `session`: A newly created transaction. Database changes executed through this object will only get written at the end of the block when the transaction is committed. If the transaction is rolled back, all the changes are discarded.

In [11]:
engine = sa.create_engine("sqlite:///chinook.db",echo=False)
with orm.Session(engine) as session:
    # query or modify the database through the session object
    pass
    # when modifying the database, commit or rollback the changes introduced in the block

## Class describing a table

This is a code pattern to describe a (row of a) single table in the database.  
This step is independent on the database (does not load the table column names/types from the database).  
This code needs to be adjusted for each table which is going to be created or modified.

Let's study the following description referring to the table `albums` from the `chinook` database:
- `Base`: This object normally is created once and it internally stores all information about the structure of the database. (*Note:* In this notebook we will recreate this object multiple times to allow changes in descriptions of the tables.)
- `Album`: This is a newly created class and it will conceptually represent a single row of the table.
- `__tablename__`: This field defines the name of the table as in the database.
- `AlbumId`: This is a name of a `Column` as in the database. Moreover, it specifies that the column keeps `Integer` numbers, and that the column belogs to the `primary_key` of this table.
- `Title`: As above, but the column keeps texts (`String` of max. 160 characters).

For more info check: [Column and Data Types](https://docs.sqlalchemy.org/en/14/core/type_basics.html).

In [12]:
Base = orm.declarative_base()

class Album(Base):
  __tablename__ = "albums" #should be exacylt the name of the table
  
  AlbumId = sa.Column(sa.Integer,primary_key=True)
  Title = sa.Column(sa.String(160))
  ArtistId = sa.Column(sa.Integer)

## Getting table rows as objects

The SQLAlchemy library provides functionality to automatically build SQL queries, execute them, and convert the results to objects of the declared tables. Study the examples below...

### SQL query is built by the library

The `session` object provides a function `query`. The arguments of this function define which table is to be asked for. Observe, how `session.query(Album)` builds automatically the SQL query based on the table description provided in the `Album` class:

In [13]:
with orm.Session(engine) as session:
    print( session.query(Album) )

SELECT albums."AlbumId" AS "albums_AlbumId", albums."Title" AS "albums_Title", albums."ArtistId" AS "albums_ArtistId" 
FROM albums


Using additionally the `limit` function we can obtain an SQL query referring to a few rows only:

In [14]:
with orm.Session(engine) as session:
    print( session.query(Album).limit(3) )

SELECT albums."AlbumId" AS "albums_AlbumId", albums."Title" AS "albums_Title", albums."ArtistId" AS "albums_ArtistId" 
FROM albums
 LIMIT ? OFFSET ?


Or, using the `filter` function we may generate SQL `where` clauses to select rows:

In [15]:
with orm.Session(engine) as session:
    print( session.query(Album).filter(Album.AlbumId == 5) )

SELECT albums."AlbumId" AS "albums_AlbumId", albums."Title" AS "albums_Title", albums."ArtistId" AS "albums_ArtistId" 
FROM albums 
WHERE albums."AlbumId" = ?


### Getting a single object

We know that `AlbumId` is the primary key of the `albums` table. So, there should be just one row for an album with (for example) `AlbumId` of `5`.  
The following code will run the SQL query and produce the `one` corresponding `Album` object:

In [16]:
with orm.Session(engine) as session:
    a = session.query(Album).filter(Album.AlbumId == 5).one()
    print(type(a))
    print(a)

# print(a)                   # the object a may be invalid once the session is closed

<class '__main__.Album'>


You can see that indeed an object of the class `Album` was produced, although its content are not shown (will be discussed next). Note though, that the values from the table columns can be accessed conveniently as the class fields:

In [17]:
with orm.Session(engine) as session:
    a = session.query(Album).filter(Album.AlbumId == 5).one()
    print(a.AlbumId)
    print(a.Title)

5
Big Ones


### Nice printing of object fields

Redefine the `Album` object a add a method `__repr__`. This is a function used to get the text representation of an object when it gets `print`ed:

In [ ]:
Base = orm.declarative_base() # normally present once in a script!

class Album(Base):
  __tablename__ = "albums"
  
  AlbumId = sa.Column(sa.Integer,primary_key=True)
  Title = sa.Column(sa.String(160))
  ArtistId = sa.Column(sa.Integer)

  def __repr__(self):
    return "Album(AlbumId='%s', Title='%s', ArtistId='%s')" % (self.AlbumId, self.Title, self.ArtistId)

Let's again get the object and print it:

In [ ]:
with orm.Session(engine) as session:
    a = session.query(Album).filter(Album.AlbumId == 5).one()
    print(a)

### Getting many objects (list)

When a query may return any number of objects use the method `.all()` instead of `.one()`. The returned value will be a list with all query results. That elements of the list will be objects representing the table rows:

In [ ]:
with orm.Session(engine) as session:
    albums = session.query(Album).limit(3).all()
    print( type( albums ) )
    print( albums )

Let's consider the first list element in more detail. Check its `type`, print it, and practice access to data from each table column:

In [ ]:
with orm.Session(engine) as session:
    albums = session.query(Album).limit(3).all()

    a = albums[1]
    print( type( a ) )
    print( a )
    print( a.Title )

### Getting many objects (for loop)

The following code might be used to access resulting objects one by one:

In [ ]:
with orm.Session(engine) as session:
  for a in session.query(Album).limit(3):
    print( a )

Or using list comprehension:

In [ ]:
with orm.Session(engine) as session:
    titles = [a.Title for a in session.query(Album).limit(3)]

print( titles ) # list of str objects, safe to use outside of the session block

### Practice



1.   Have a look into [the documentation](https://docs.sqlalchemy.org/en/14/orm/tutorial.html#returning-lists-and-scalars) and understand the functions: `one()`, `first()`, `all()`, `one_or_none()`.
1.   Calculate the number of `Album`s in the database. Construct the query and use the [`count()`](https://docs.sqlalchemy.org/en/14/orm/tutorial.html#counting) function.



## Another class describing a related table

### Class (partially) describing another table

In [ ]:
Base = orm.declarative_base() # normally present once in a script!

class Album(Base):
  __tablename__ = "albums"
  
  AlbumId = sa.Column(sa.Integer,primary_key=True)
  Title = sa.Column(sa.String(160))
  ArtistId = sa.Column(sa.Integer)

  def __repr__(self):
    return "Album(AlbumId='%s', Title='%s', ArtistId='%s')" % (self.AlbumId, self.Title, self.ArtistId)

class Track(Base):
  __tablename__ = "tracks"

  TrackId = sa.Column(sa.Integer,primary_key=True)
  Name = sa.Column(sa.String(200))
  AlbumId = sa.Column(sa.Integer)

  def __repr__(self):
    return "Track(TrackId='%s', Name='%s', AlbumId='%s')" % (self.TrackId, self.Name, self.AlbumId)

### Observing a relation between two tables

Check in the database diagramm:
- an `Album` refers to zero or more tracks
- a `Track` refers to zero or one album (through the `AlbumId` foreign key)


Let's study the `Album` with `AlbumId` of 5:

In [ ]:
with orm.Session(engine) as session:
    a = session.query(Album).filter(Album.AlbumId == 5).one()
    print(a)

These are the `Track`s which refer to this album:

In [ ]:
with orm.Session(engine) as session:
  for t in session.query(Track).filter(Track.AlbumId == 5).all():
    print(t)

### Generating methods representing relations

From the object oriented programming point of view it would be useful to:
- Have a method `album()` in the class `Track` which returns the `Album` object corresponding to the `AlbumId` of the track.
- Have a method `tracks()` in the class `Album` which returns the list of `Track` objects referring to this album.

To define such a relation the following two changes are needed (look at the code below):
- In the `Track` class:

  the field `AlbumId` gets declared as a `ForeginKey` to `albums.AlbumId` (so a foreign key to the table `albums` where the key is in column `AlbumId`). With `nullable` you can specify whether it is allowed that a `Track` does not refer to any `Album`.
- In the `Album` class:

  the field `tracks` gets declared as a relationship to the class `Track`. In the class `Track` the reverse relationship should be called `album`.

See [relationship patterns](https://docs.sqlalchemy.org/en/14/orm/basic_relationships.html#relationship-patterns) for more details.

In [ ]:
Base = orm.declarative_base() # normally present once in a script!

class Album(Base):
  __tablename__ = "albums"
  
  AlbumId = sa.Column(sa.Integer,primary_key=True)
  Title = sa.Column(sa.String(160))
  ArtistId = sa.Column(sa.Integer)
  tracks = orm.relationship("Track", backref="album")                 # <<<< HERE

  def __repr__(self):
    return "Album(AlbumId='%s', Title='%s', ArtistId='%s')" % (self.AlbumId, self.Title, self.ArtistId)

class Track(Base):
  __tablename__ = "tracks"

  TrackId = sa.Column(sa.Integer,primary_key=True)
  Name = sa.Column(sa.String(200))
  AlbumId = sa.Column(sa.ForeignKey('albums.AlbumId'), nullable=True) # <<<< HERE

  def __repr__(self):
    return "Track(TrackId='%s', Name='%s', AlbumId='%s')" % (self.TrackId, self.Name, self.AlbumId)

Let's get an `Album` object `a` now and see the corresponding `Track` objects through the `tracks` method:

In [ ]:
with orm.Session(engine) as session:
    a = session.query(Album).filter(Album.AlbumId == 5).one()
    print(a)
    print(a.tracks)

## Adding objects with new data to database

Let's consider the goal of adding a new `Album` to the database.  
*Note:* (check on the database diagramm) that each album must refer to exactly one artist. So it is impossible to add an album without a proper reference to an artist.  
Consequently, we first need to properly describe the relationship between `albums` and `artists` tables.  
Then, we will choose a random artist and add a new fictional album of that artist.

### Another table, another relationship

The following modifications are made here (try yourself to introduce these modifications without looking into the code below):
- New class `Artist` describing the `artists` table
- The `Album` gets updated to have a proper foreign key to `Artist`
- The `Artist` gets a relationship to `Album` through fields: `Artist.albums` and `Album.artist`.

In [ ]:
Base = orm.declarative_base() # normally present once in a script!

class Album(Base):
  __tablename__ = "albums"
  
  AlbumId = sa.Column(sa.Integer,primary_key=True)
  Title = sa.Column(sa.String(160))
  ArtistId = sa.Column(sa.ForeignKey('artists.ArtistId'), nullable=False) # <<<< HERE
  tracks = orm.relationship("Track", backref="album")

  def __repr__(self):
    return "Album(AlbumId='%s', Title='%s', ArtistId='%s')" % (self.AlbumId, self.Title, self.ArtistId)

class Track(Base):
  __tablename__ = "tracks"

  TrackId = sa.Column(sa.Integer,primary_key=True)
  Name = sa.Column(sa.String(200))
  AlbumId = sa.Column(sa.ForeignKey('albums.AlbumId'), nullable=True)

  def __repr__(self):
    return "Track(TrackId='%s', Name='%s', AlbumId='%s')" % (self.TrackId, self.Name, self.AlbumId)

class Artist(Base):
  __tablename__ = "artists"

  ArtistId = sa.Column(sa.Integer,primary_key=True)
  Name = sa.Column(sa.String(120))
  albums = orm.relationship("Album", backref="artist")                  # <<<< HERE

  def __repr__(self):
    return "Artist(ArtistId='%s', Name='%s')" % (self.ArtistId, self.Name)


Let's check the new Artist-Album relationship:

In [ ]:
with orm.Session(engine) as session:
    print( session.query(Artist).count() )        # total number of artists in the database
    print( session.query(Artist).first() )        # first artist
    print( session.query(Artist).first().albums ) # albums of the first artist

### Adding a new object (related to existing data/objects)

The lifetime of the `Session` object defines a transaction.  
When changes to table/row objects are made they are not written immediately.  
They get written when `commit()` method of the `Session` object is called.  
Note, that if multiple session objects exist simultaneously, the data between them are not synchronized.  
Consequently, checking newly added (modified) content after `commit()` cannot be done through another session object which was created before `commit()` was called.

Let's list the `Album`s of Metallica existing in the database:

In [ ]:
with orm.Session(engine) as session:
  artist = session.query(Artist).filter(Artist.Name=="Metallica").one()
  print(artist, "\n") 
  for album in artist.albums: 
    print( album )

Here is an example of adding a new `Album`.  
Note, that the `AlbumId` will get automatically generated and that the `ArtistId` will be automatically collected when `theArtist` is assigned to the `artist` relationship field.  
The changes will get written to the database the moment of `commit()`.

In [ ]:
with orm.Session(engine) as session:
  artist = session.query(Artist).filter(Artist.Name=="Metallica").one()

  newAlbum = Album(Title='Hardwired... to Self-Destruct') # new Album object, only in memory
  newAlbum.artist = artist                                # building relation to theArtist
  print(newAlbum)

  session.add(newAlbum)                                   # staging Album to become written to the database
  session.commit()                                        # writing to the database
  
  print(newAlbum)

Let's check whether the new album has been added:

In [ ]:
with orm.Session(engine) as session:
  artist = session.query(Artist).filter(Artist.Name=="Metallica").one()
  print(artist, "\n") 
  for album in artist.albums: 
    print( album )

## Multistep exercise

The final goal of this session is to build a class providing access to a database.  
This class should provide all needed database content through a well defined interface.  

Follow the points below in the provided order (later points depend on the earlier ones).  
Try to implement each step without checking the solution provided below.  
Try to type. Do not copy-paste.

The solution is suboptimal - the returned objects are not fully independent of the current session and they may provide inconsistent data.

1. Define a new class `Chinook`. This class will contain all methods to access the database. The methods will be added gradually.
1. In the class `Chinook`: 
    - Define the class constructor `__init__(self, url)`.
    - The `url` argument will provide the location of the database as needed by the `create_engine` function.
1. In the class `Chinook`:
    - In the constructor create the engine and store it in the attribute `self._engine`. 
1. Outside the class:
    - Create an object instance of the class `Chinook`. 
    - Use the proper `url` argument (so it uses the `chinook.db` file).
    - Store the new object in the variable `db`.
1. In the class `Chinook`:
    - Define a new method `addArtist(self, name)`. 
    - The method should add a new artist of `name` to the database. 
    - Perform adding within a new local session. 
    - Remember to call `commit` in the session. 
    - The function should return the value of `ArtistId` of the newly added artist.
1. In the class `Chinook`:
    - Define a new method `getArtist(self, artistId)`. 
    - The method should return an `Artist` object for the provided `artistId`.
    - (Returning the `Artist` object is not the best solution because it would exist outside the session; but it is good enough for now.)
1. Outside the class:
    - Recreate the object in `db` with the updated class. 
    - Write the code to `addArtist` with a randomly chosen name.
    - Store the returned `artistId` value in a variable.
    - Check whether you can get the artist back with `getArtist`.
1. In the class `Chinook`:
    - Define a new method `allArtists(self)`.
    - The method should return a list of all `Artist` objects present in the database.
    - (Returning the list of `Artist` objects is not the best solution because it would exist outside the session; but it is good enough for now.)
1. Outside the class:
    - Recreate the object in `db` with the updated class. 
    - Use the result of `db.allArtists()` in a `for` loop to print all artists from the database.
    - Find how to print only the last 10 of the returned objects.
1. Outside the class:
    - In `firstNames` create a vector with several random popular first names.
    - Similar for `surNames`.
    - From package `random` use `choice` method. It randomly selects an element from a list.
    - Create a random name by concatenating a random first name and a random surname from your lists.
    - Insert above into a loop which adds to the database 10 artists with randomly generated names.
    - Print last 10 artists to check whether they were indeed added.


In [ ]:
class Chinook:
  def __init__(self, url):
    self._engine = sa.create_engine(url=url,echo=False)

  def addArtist(self, name):
    with orm.Session(self._engine) as session:
      a = Artist(Name=name)
      session.add(a)
      session.commit()
      return a.ArtistId # here a number is returned, no Session object needed

  def getArtist(self, artistId):
    with orm.Session(self._engine) as session:
      a = session.query(Artist).where(Artist.ArtistId == artistId).one()
    return a # this is not safe; the object exists outside the session object; relations will fail

  def allArtists(self):
    with orm.Session(self._engine) as session:
      artists = session.query(Artist).all()
    return artists # this is not safe; the object exists outside the session object; relations will fail

db = Chinook("sqlite:///chinook.db") # this should be used inside "with" context manager
aId = db.addArtist(name="The Singer")
print( "The ArtistId of the new artist is: ", aId)
print( db.getArtist(artistId=aId))

from random import choice

firstNames = [ "John", "Johan", "Jan", "Ivan" ]
lastNames = [ "Smith", "Kowalski", "Kovalsky" ]

for i in range(5):
  n = choice(firstNames) + " " + choice(lastNames)
  db.addArtist(name=n)

for a in db.allArtists()[-8:-1]:
  print(a)